In [1]:
import sys
sys.path.append('./')

In [2]:
model_ckpt = "MCG-NJU/videomae-base"


In [3]:
import torch
import torch.nn as nn

from transformers import VideoMAEModel 
class my_model(nn.Module):
    def __init__(self):
        super().__init__()#"MCG-NJU/videomae-base")
        self.model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
        self.average = nn.AdaptiveAvgPool1d(1)
        #self.linear =nn.Linear(768,128)
        #self.norm =nn.functional.normalize() 
    def forward(self, inputs):
        
        x = self.model(inputs).last_hidden_state
        #print('1:',x.shape)
        x = x.permute(0,2,1)
        #print('2:',x.shape)
        x = self.average(x).squeeze()
        #x = self.linear(x)

        #self.std = torch.mean(x)
        #self.var = torch.var(x)
        #x = self.norm(x)
        x = nn.functional.normalize(x)
        #print('3:',x.shape)
        return x


2023-01-11 09:21:13.354364: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 09:21:16.697824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-11 09:21:16.698096: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-11 09:21:16.698110: W tensorfl

In [4]:
#norm = nn.functional.normalize(x)

In [4]:
model = my_model()
model.cuda()

Some weights of the model checkpoint at MCG-NJU/videomae-base were not used when initializing VideoMAEModel: ['decoder.decoder_layers.3.output.dense.weight', 'decoder.decoder_layers.0.attention.attention.key.weight', 'decoder.head.weight', 'decoder.decoder_layers.0.layernorm_before.weight', 'decoder.decoder_layers.3.attention.attention.query.weight', 'decoder.decoder_layers.3.layernorm_before.weight', 'decoder.decoder_layers.1.output.dense.bias', 'decoder.decoder_layers.3.layernorm_before.bias', 'decoder.head.bias', 'decoder.decoder_layers.1.layernorm_after.bias', 'decoder.decoder_layers.3.attention.attention.key.weight', 'decoder.decoder_layers.2.attention.attention.query.weight', 'decoder.decoder_layers.1.intermediate.dense.weight', 'decoder.decoder_layers.1.layernorm_before.weight', 'decoder.decoder_layers.3.attention.attention.v_bias', 'decoder.decoder_layers.0.intermediate.dense.bias', 'decoder.decoder_layers.3.attention.output.dense.weight', 'decoder.decoder_layers.3.attention.at

my_model(
  (model): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0): VideoMAELayer(
          (attention): VideoMAEAttention(
            (attention): VideoMAESelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
            (dense): Linear(in_features=768, out_features

In [5]:
#parallel_net = nn.DataParallel(model, device_ids = [0,1])

parallel_net = nn.DataParallel(model, device_ids = [0,1,2,3,4,5,6,7])

In [6]:
import numpy as np
from utils.dataload import CustomTensorDataset,CustomTensorDatasetTrain


In [7]:
big_tensor_video_train = np.load('train_video_letters_only_data_2.npy')
big_tensor_video_test = np.load('test_video_letters_only_data_2.npy')


In [9]:
big_tensor_video_train.shape

(64, 16, 3, 224, 224)

In [10]:
big_tensor_video_test.shape

(32, 16, 3, 224, 224)

In [11]:
big_tensor_video_train.shape

(64, 16, 3, 224, 224)

In [12]:
big_tensor_video_test.dtype

dtype('float64')

In [13]:
train_dataset = CustomTensorDatasetTrain(big_tensor_video_train)
test_dataset = CustomTensorDataset(big_tensor_video_test)

In [14]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset,batch_size=64,shuffle = False, num_workers=1)  
test_dataloader = DataLoader(test_dataset,batch_size = 32)  

In [15]:
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.reducers import ThresholdReducer
from pytorch_metric_learning.regularizers import LpRegularizer
from pytorch_metric_learning import losses
#loss_func = losses.TripletMarginLoss(distance = CosineSimilarity(), 
#                                    reducer = ThresholdReducer(high=0.3), 
#                                    embedding_regularizer = LpRegularizer()
#                                   )
from pytorch_metric_learning import miners

In [16]:
#miner = miners.TripletMarginMiner()
#loss_func = losses.TripletMarginLoss()


In [17]:
from pytorch_metric_learning import miners, losses
miner = miners.TripletMarginMiner(margin=0.5, type_of_triplets="semihard")
loss_func = losses.TripletMarginLoss(margin=0.5)



In [18]:
from torch.optim import Adam
optimizer = Adam(model.parameters(),lr =0.00001)

In [19]:

from utils.dimension_reduction import run_pca
from utils.plot import plot_embeddings

In [20]:
#epochs = 3
#for epoch in range(epochs):
#    print('epoch:',epoch)    
#    for x,y in train_dataloader:
#        
#        print('y:',y)
#
#    train_dataset.shuffle()


In [21]:
config =5 # to be read from the file
dim = 3

In [22]:
from sklearn.metrics import confusion_matrix

def get_distances(embeddings):
    print(embeddings.shape)
    distances = np.zeros((embeddings.shape[0],embeddings.shape[0]))
    for i in range(embeddings.shape[0]):
        for j in range(embeddings.shape[0]):
            #a = embeddings[i]
            #b = embeddings[j]
            #print(a.shape)
            #print(b.shape)
           
            distances[i][j] = np.linalg.norm(embeddings[i]-embeddings[j])
    return distances

def get_prediction(embeddings,data):
    label = 0
    min_distance = 10000
    for k in range(len(embeddings)):
        #dist = torch.dist(embeddings[k],data,2)
        dist = np.linalg.norm(embeddings[k] - data)
        

        if min_distance>dist :

            min_distance = dist 
            label = k
    return label


def calculate_acc(train_embeddings,train_labels, test_embeddings, test_labels):
    accuracy = 0
    labels = []
    for i in range(test_embeddings.shape[0]):
        label = train_labels[get_prediction(train_embeddings,test_embeddings[i])]
        labels.append(label)
    print('labels:',labels)#,test_labels.tolist())
    conf_mat = confusion_matrix(labels, test_labels.tolist(),labels=np.arange(32))
    accuracy = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
    return accuracy ,conf_mat 

def calculate_acc_train(train_embeddings,train_labels):
    accuracy = 0
    distances = get_distances(train_embeddings)
    for i in range(distances.shape[0]):
        print(train_labels[i],train_labels[np.argsort(distances[i])][:10])



def calculate_acc_all(train_embeddings,train_labels,test_embeddings,test_labels):
    distances = get_distances(np.concatenate((train_embeddings,test_embeddings),axis =0) )
    for i in range(distances.shape[0]):
        print(np.concatenate((train_labels,test_labels), axis = 0)[i],np.concatenate((train_labels,test_labels),axis = 0)[np.argsort(distances[i])][:])
    

In [23]:
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def evaluate_test():
    outputs = []
    labels_list = []
    with torch.no_grad():
        for i,(j,k) in enumerate(test_dataloader):
            input = j.cuda()#.permute(0,2,1,3,4).cuda()
            embeddings = parallel_net(input)#.logits
            #print('test_shape:',embeddings.shape)
           
            #labels = k.type(torch.int64).cuda()
            #print('test_label:',k.numpy())
            outputs.append(embeddings.cpu().numpy())
            labels_list.append(k.numpy())
        test_embeddings, test_labels = np.vstack(outputs), np.concatenate(labels_list).astype(int)
        #print('test_labels:',test_labels)
    outputs = []
    labels_list = []
    with torch.no_grad():
        for i,(j,k) in enumerate(train_dataloader):
            input = j.cuda()#.permute(0,2,1,3,4).cuda()
            embeddings = parallel_net(input)#.logits
            #print('test_shape:',embeddings.shape)
            
            #labels = k.type(torch.int64).cuda()

            outputs.append(embeddings.cpu().numpy())
            labels_list.append(k.numpy())
        train_embeddings, train_labels = np.vstack(outputs), np.concatenate(labels_list).astype(int)
    calculate_acc_all(train_embeddings,train_labels,test_embeddings,test_labels)
    
    accuracy,conf_mat = calculate_acc(train_embeddings,train_labels, test_embeddings, test_labels)

    #cmd = ConfusionMatrixDisplay(conf_mat)
    #cmd.plot()
    print('Test:', accuracy)

In [24]:
import numpy as np
def run_test():
    outputs = []
    labels_list = []
    with torch.no_grad():
        for i,(j,k) in enumerate(test_dataloader):
            input = j.type(torch.FloatTensor).cuda()#.permute(0,2,1,3,4).cuda()
            embeddings = parallel_net(input)#.logits
            #print('test_shape:',embeddings.shape)

            
            labels = k.type(torch.int64).cuda()
            hard_pairs= miner(embeddings,labels)
            loss = loss_func(embeddings, labels , hard_pairs)
            #print('test:',loss.item())


            outputs.append(embeddings.cpu().numpy())
            labels_list.append(k.numpy())
    return np.vstack(outputs), np.concatenate(labels_list)

In [25]:
def predict(predicts,corpus):
    pass

In [26]:
from utils.utils import createFolder

In [27]:
epochs = 200

In [28]:
config =12

In [29]:
createFolder('/projects/plots/exp'+str(config), delete=True)
for epoch in range(epochs):
    outputs = []
    labels_list = []
    big_loss = 0
    print('epoch:',epoch)
    for i,(j,k) in enumerate(train_dataloader):

        #start = time.time()
        input = j.type(torch.FloatTensor).cuda()#.permute(0,2,1,3,4).cuda()
        embeddings = parallel_net(input)#.logits
        #print('embed:',embeddings)
        #print(embeddings.shape)
        labels = k.type(torch.int64).cuda()
        #print(input.dtype,labels.dtype)
        #print('label:',labels.T)

        #print(embeddings.shape, labels.shape)
        hard_pairs= miner(embeddings,labels)
        
        print('hard_pairs:',len(hard_pairs[0]))
        #print('hard_pairs:',hard_pairs)
        loss = loss_func(embeddings, labels , hard_pairs)
        loss.backward()
        big_loss += loss.item()
        #print(loss.item())
        optimizer.step()
        optimizer.zero_grad()
        
        outputs.append(embeddings.detach().cpu().numpy())
        labels_list.append(k.detach().numpy())
    print('loss:',big_loss/4)

    train_dataset.shuffle()
    #print('ddL:',outputs[0].shape)
    #print('dddd:',len(labels_list))
    embeddings, labels = run_test()
    if epoch%4==0:
        evaluate_test()
    outputs.append(embeddings)
    labels_list.append(labels)
    labels_new = np.concatenate(labels_list).squeeze()

    #print('qqq:',np.vstack(outputs).shape)
    
    a = run_pca(np.vstack(outputs), dim)
    #print(a.shape)
    #print(labels_new)
    labels_new = list(map(int, labels_new))

    plot_embeddings(a,labels_new,epoch,config)
    #outputs.append(y.detach().cpu().numpy())
    #labels_list.append(j['label'].numpy())

epoch: 0
hard_pairs: 3968
loss: 0.09076178073883057
(96, 768)
14 [14 14  7 11 22 22 13 26 26 19 29 24 20 28 28 14 23 21 18 18  9 22 27 28
 16 16  8 17 24 24 12 30 30 15 18  3  3  5  5 19 19 29 29 31 31 27 27 15
 15  9  9  7  7 23 23 25 25 13 13 17 17 11 11  1  1 21 21  8  8  4  2  2
  1  0  0  0 30 25  4  2  4 16 31 26  3  6  6  6 12 12 20 20 10 10  5 10]
14 [14 14  7 11 22 22 13 26 26 19 29 24 20 28 28 14 23 21 18 18  9 22 27 28
 16 16  8 17 24 24 12 30 30 15 18  3  3  5  5 19 19 29 29 31 31 27 27 15
 15  9  9  7  7 23 23 25 25 13 13 17 17 11 11  1  1 21 21  8  8  4  2  2
  1  0  0  0 30 25  4  2  4 16 31 26  3  6  6  6 12 12 20 20 10 10  5 10]
25 [25 25 31 31 11 11 17 17 13 13  5  5  7  7  9  9 15 15 27 27  3  3 23 23
  1  1 21 21 19 19 29 29 18 27 21 24 24 12 29 28 17 15 30 30 18 18  9 13
 26 26 23 16 16  8 22 22 11 22 14 14  7 20 24 19 14 28 28  4  8  8  0  0
  0  1  2  2 30 25 16 31  2  4  4 26  6 12 12 20 20 10  3  6  6  5 10 10]
25 [25 25 31 31 11 11 17 17 13 13  5  5  7  7  9  

In [ ]:
torch.save(model.state_dict(), '/projects/models/Triplet/model_config_'+str(config)+'_'+str(epochs)+'_epoch.pth')

In [ ]:
from pytorch_metric_learning import testers
tester = testers.BaseTester()
dataset_dict = {"train": train_dataset, "val": test_dataset}
#all_accuracies = tester.test(dataset_dict, epoch, parallel_net)
